# Libraries

In [2]:
import pyodbc
import pandas as pd
from openpyxl import load_workbook

# Database settings

In [3]:
# Connect to the database
server = r'NOLTE01\CHARLESNOLTE'
database = 'SouthAfrica_CrimeStats'
username = 'cnoltehj'
password = 'HeidiJoy@1'


# 2016-2020 Validate Policestation and Exclude rows

In [112]:
# Load the Excel file
# file_path = "./Dataset_CSV_Files/2020_2021/first_quarter 2020_2021_crime_statistics.xlsx"
# Quarter = 1

file_path = "./Dataset_CSV_Files/2024_2025/2024-2025_-_2nd_Quarter_WEB.xlsx"
Quarter = 2

# file_path = "./Dataset_CSV_Files/2024_2025/2024_2025_3rd_Quarter_WEB.xlsx"
# Quarter = 3

# file_path = "./Dataset_CSV_Files/2024_2025/2024-2025_Q4_crime_stats.xlsx" 
# Quarter = 4

wb = load_workbook(file_path, data_only=True)

# Access the desired sheet
# Q-1, 2,3,4
sheet = wb["Dropdown tables stations"] #wb["Stations per province"] --- 2024 : Crime stats per component

# Quarter
#sheet = wb["Crime stats per component"]

# Check the value of cell Q8
cell_J9 = sheet["J9"].value

# Check the value of cell Q9
cell_J10 = sheet["J10"].value

# 1 Quarter
# excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 28, 29, 36, 40, 47, 53]

# 2 Quarter
excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 28, 35, 39, 46, 52]

#3 Quarter and 4 Quarter
#excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 29, 36, 40, 47, 53]

excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]  # First 2 columns and last 2 columns

print("Quarter : ",Quarter)
print("Province : ",cell_J9)
print("Police Station : ",cell_J10)

# #     # Iterate over all rows except for the excluded ones
# for row_number in range(2, sheet.max_row + 1):
#     if row_number not in excluded_rows:
#             # Retrieve row values and exclude specified columns
#         row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]

#             # Print the row values as a single dataset
#         print(row_values)

KeyError: 'Worksheet Dropdown tables stations does not exist.'

# 2016-2020 Insert Stats into Database Omit 2020

In [ ]:
cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()

# Load workbook
file_path = './Dataset_CSV_Files/2020_2021/Q2_2016.xlsx'
wb = load_workbook(file_path, data_only=True)

# Loop through sheet1 - sheet150
for i in range(153, 154):
    sheet_name = f"Sheet{i}"
    
    if sheet_name not in wb.sheetnames:
        print(f"Skipping {sheet_name} (does not exist)")
        continue
    
    sheet = wb[sheet_name]
    print(f"\nProcessing {sheet_name}...")

    # Get header info
    cell_J9 = sheet["J9"].value
    cell_J10 = sheet["J10"].value
    selected_province = cell_J9
    selected_policestation = 'Simons Town' #cell_J10
    selected_quarter = 2

    print(f"Quarter: {selected_quarter}, Province: {selected_province}, Police Station: {selected_policestation}")

    excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 28, 35, 39, 46, 52] #Q2

    #excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 28, 29, 36, 40, 47, 53] # Q3 / Q4

    excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]

    # Process each data row
    for row_number in range(3, sheet.max_row + 1):
        if row_number in excluded_rows:
            continue

        row_values = [
            cell.value for col_num, cell in enumerate(sheet[row_number])
            if col_num + 1 not in excluded_columns
        ]

        # Skip empty or invalid rows
        if all(v in (None, "", " ") for v in row_values):
            continue

        try:
            cursor.execute(
                "EXEC sp_Add_QuarterlyCrimeStats_2016_2019 ?, ?, ?, ?, ?, ?, ?, ?",
                (
                    str(row_values[0]).strip(),
                    row_values[1],
                    row_values[2],
                    row_values[3],
                    row_values[4],
                    str(selected_province).strip(),
                    str(selected_policestation).strip(),
                    int(selected_quarter)
                )
            )
            cnxn.commit()
            print(f"Inserted row {row_number} from {sheet_name}")

        except pyodbc.Error as e:
            print(f"SQL Error in {sheet_name} row {row_number}: {e}")

# Cleanup
cursor.close()
cnxn.close()
wb.close()
print("\n Processing complete for all sheets.")


Processing Sheet153...
Quarter: 2, Province: WESTERN_CAPE, Police Station: Simons Town
Inserted row 14 from Sheet153
Inserted row 15 from Sheet153
Inserted row 16 from Sheet153
Inserted row 17 from Sheet153
Inserted row 18 from Sheet153
Inserted row 19 from Sheet153
Inserted row 20 from Sheet153
Inserted row 21 from Sheet153
Inserted row 23 from Sheet153
Inserted row 24 from Sheet153
Inserted row 25 from Sheet153
Inserted row 26 from Sheet153
Inserted row 27 from Sheet153
Inserted row 29 from Sheet153
Inserted row 30 from Sheet153
Inserted row 31 from Sheet153
Inserted row 32 from Sheet153
Inserted row 33 from Sheet153
Inserted row 34 from Sheet153
Inserted row 36 from Sheet153
Inserted row 37 from Sheet153
Inserted row 38 from Sheet153
Inserted row 40 from Sheet153
Inserted row 41 from Sheet153
Inserted row 42 from Sheet153
Inserted row 43 from Sheet153
Inserted row 44 from Sheet153
Inserted row 45 from Sheet153
Inserted row 47 from Sheet153
Inserted row 48 from Sheet153
Inserted row

: 

# Insert Trio Crime

In [ ]:

cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()

# === 1. Fetch all police station codes ===
cursor.execute("EXEC sp_Get_All_PoliceStationCodes")
stations = cursor.fetchall()
print(f"Total stations found: {len(stations)}")

# === 2. Loop through each station ===
for row in stations:
    station_code = row[0]  # Extract the string value (e.g. 'AL01WC')
    print(f"Processing StationCode: {station_code}")

    try:
        # Pass parameter correctly as a tuple (note the comma)
        cursor.execute("EXEC sp_Update_Q4TrioCrime @PoliceStationCode = ?", (station_code,))
        cnxn.commit()
        print(f"Successfully updated: {station_code}")

    except Exception as e:
        print(f"Error updating {station_code}: {e}")

# === Cleanup ===
cursor.close()
cnxn.close()
print("All stations processed successfully.")

Total stations found: 152
Processing StationCode: AL01WC
✅ Successfully updated: AL01WC
Processing StationCode: AS02WC
✅ Successfully updated: AS02WC
Processing StationCode: AT03WC
✅ Successfully updated: AT03WC
Processing StationCode: ATL04WC
✅ Successfully updated: ATL04WC
Processing StationCode: BA05WC
✅ Successfully updated: BA05WC
Processing StationCode: BD14WC
✅ Successfully updated: BD14WC
Processing StationCode: BE07WC
✅ Successfully updated: BE07WC
Processing StationCode: BF13WC
✅ Successfully updated: BF13WC
Processing StationCode: BG12WC
✅ Successfully updated: BG12WC
Processing StationCode: BL010WC
✅ Successfully updated: BL010WC
Processing StationCode: BO11WC
✅ Successfully updated: BO11WC
Processing StationCode: BS09WC
✅ Successfully updated: BS09WC
Processing StationCode: BV08WC
✅ Successfully updated: BV08WC
Processing StationCode: BW06WC
✅ Successfully updated: BW06WC
Processing StationCode: CB17WC
✅ Successfully updated: CB17WC
Processing StationCode: CD15WC
✅ Success